In [13]:
!python --version

Python 3.9.16


In [2]:
conda env list

# conda environments:
#
base                     /opt/conda
text-embedding           /opt/conda/envs/text-embedding


Note: you may need to restart the kernel to use updated packages.


## テキストクリーニング

In [21]:
!pip install beautifulsoup4

In [20]:
!pip install neologdn

  Using cached neologdn-0.5.2.tar.gz (86 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for neologdn: filename=neologdn-0.5.2-cp310-cp310-linux_x86_64.whl size=56749 sha256=02618e99e0bf45517394f62715726952b354982e4d0b02a4529db343e03e5928
  Stored in directory: /root/.cache/pip/wheels/bd/87/af/2a36d74f666a8428943b70d71c5e9dd740435bf671f210672c
Successfully built neologdn


In [2]:
import pandas as pd
import re
import os

from bs4 import BeautifulSoup
import neologdn

In [3]:
# load dataset
home = os.environ.get('HOME')
df = pd.read_table(f'{home}/data/qiita_2018.tsv', low_memory=False)
# 負荷調整
# df = df[:10]

# 指定カラムのみ取り出す
usecols = ['title', 'body', 'tags']
df = df[usecols]
df

,title,body,tags
0,iOS Simulatorでシミュレータのsqlite dbを参照する方法,お初になります。新人えんじにゃーの@marloweです。\n\nタイトル通り、iOSアプリの...,Xcode iOS
1,シンギュラリティは近い ～全人類がいますぐ注目すべき特異点論と多目的最適化の知られざる関係～,# はじめに\n[Fujitsu Advent Calendar 2017](https:/...,数学 最適化 シンギュラリティ
2,IoT開発の利器、pi-sync,# IoT開発の利器、pi-sync\n\n## 前書き\n __あけましておめでとうござ...,Node.js RaspberryPi npm IoT
3,C#でもBLASとかLAPACKを使えるようにした！一部だけ！,# はじめに\n\n**新年、あけましておめでとうございます！**\n\n初投稿になります！...,C# C++CLI mkl 速度比較 .NETCore
4,C++でマークシート読み取れるフリーソフトを自作してみた！,初めて投稿させていただきます。個人の作成記録のメモですが、記事中に出てくる疑問点に関してコメ...,PHP C++ C# OpenCV Conoha
...,...,...,...
90308,IPv4 マルチキャストによるフェイルオーバー,## 1. 概要\nこの記事では、IPv4 マルチキャストによるルーティングフェイルオーバー...,Go Linux Network multicast
90309,充電用のMicroUSBケーブルを見つけるデバイス #大晦日ハッカソン,これは [#大晦日ハッカソン](https://atnd.org/events/102652...,USB 大晦日ハッカソン
90310,LuaスクリプトからNetBSD Luaカーネルモジュールの値を参照できるようにする,またしても遅ればせながら [NetBSD Advent Calendar 2018](htt...,Lua AdventCalendar NetBSD
90311,【Python】近くの喫茶店・カフェを教えてくれる LINE BOT作成の記録・解説,## Introduction\n Pythonの勉強をしながら、折角だから何か形になるもの...,Python Heroku Python3 linebot LINEmessagingAPI


In [4]:
# cleaning function
# @see https://github.com/Hironsan/natural-language-preprocessings/blob/master/preprocessings/ja/cleaning.py
def clean_text(text):
    if '\n' in text:
        replaced_text = '\n'.join(s.strip() for s in text.splitlines()[2:] if s != '')  # skip header by [2:]
    else:
        replaced_text = text # no replace
    replaced_text = neologdn.normalize(replaced_text) # neologdnによる表記ゆれ統一
    replaced_text = replaced_text.lower() # 小文字変換
    replaced_text = re.sub(r'[【】]', ' ', replaced_text)       # 【】の除去
    replaced_text = re.sub(r'[（）()]', ' ', replaced_text)     # （）の除去
    replaced_text = re.sub(r'[［］\[\]]', ' ', replaced_text)   # ［］の除去
    replaced_text = re.sub(r'[@＠]\w+', '', replaced_text)  # メンションの除去
    replaced_text = re.sub(r'https?:\/\/.*?[\r\n ]', '', replaced_text)  # URLの除去
    replaced_text = re.sub(r'[!-/:-@[-`{-~]', r' ', replaced_text) # 半角記号の置き換え
    replaced_text = re.sub(r'\d+', '0', replaced_text) #数値を0に置換
    replaced_text = re.sub(r'　', ' ', replaced_text)  # 全角空白の除去
    return replaced_text


def clean_html_tags(html_text):
    soup = BeautifulSoup(html_text, 'html.parser')
    cleaned_text = soup.get_text()
    cleaned_text = ''.join(cleaned_text.splitlines())
    return cleaned_text


def clean_html_and_js_tags(html_text):
    soup = BeautifulSoup(html_text, 'html.parser')
    [x.extract() for x in soup.findAll(['script', 'style'])]
    cleaned_text = soup.get_text()
    cleaned_text = ''.join(cleaned_text.splitlines())
    return cleaned_text


def clean_url(html_text):
    """
    \S+ matches all non-whitespace characters (the end of the url)
    :param html_text:
    :return:
    """
    clean_text = re.sub(r'http\S+', '', html_text)
    return clean_text


def clean_code(html_text):
    """Qiitaのコードを取り除きます
    :param html_text:
    :return:
    """
    soup = BeautifulSoup(html_text, 'html.parser')
    [x.extract() for x in soup.findAll(class_="code-frame")]
    cleaned_text = soup.get_text()
    cleaned_text = ''.join(cleaned_text.splitlines())
    return cleaned_text

def clean_all(text):
    text = clean_text(text)
    text = clean_html_tags(text)
    text = clean_html_and_js_tags(text)
    text = clean_url(text)
    text = clean_code(text)
    return text

# exec cleaning 
df.title = df.title.apply(clean_text)
df.body = df.body.apply(clean_all)

In [45]:
df.body

0    タイトル通り、iosアプリの開発をしていて、ローカルdbの中身を確認したくなった時に使ってま...
1    さて これから解説するシンギュラリティは 0年に人工知能がどうこうという話ではなく 数学にお...
2      前書き  あけましておめでとうございます  。0年の初qiitaです。よろしくお願いいた...
3      新年、あけましておめでとうございます   初投稿になります 皆様はじめまして、rafka...
4    マークシート等を読み取れるcell sheetというソフトを作成してみました。こんな感じのシ...
5     url     シェルを取得 shell   new object  comobject ...
6    ※osはwindows0 0bitです。   angular開発環境の構築    node ...
7      使ったもの mac book air usbマウス microbit usbケーブル タ...
8    ファイルシステムとは、データとメタデータをある構造に従ってストレージデバイス上に格納したもの...
9    macのプレビュー appがちょっとだけ嫌い。ページめくりを space  or  shif...
Name: body, dtype: object

In [46]:
df.title

0          ios simulatorでシミュレータのsqlite dbを参照する方法
1    シンギュラリティは近い全人類がいますぐ注目すべき特異点論と多目的最適化の知られざる関係
2                               iot開発の利器、pi sync
3                c でもblasとかlapackを使えるようにした 一部だけ 
4                  c  でマークシート読み取れるフリーソフトを自作してみた 
5               powershellでinternetexplorerを操作する
6             angular cliでプロジェクトを作成し、githubで管理する
7                            0歳にプログラミングを教えて失敗した話
8                        知っておきたいlinuxファイルシステムの概念
9               kensington プレビュー appのページめくりを楽にする
Name: title, dtype: object

C# など若干本来の意味が削られている部分もあるが，一旦許容する

In [40]:
df

,coediting,comments_count,created_at,id,likes_count,organization_url_name,page_views_count,private,reactions_count,slide,stocks_count,tags,team_membership,title,updated_at,url,user
0,False,0,2018-01-01 00:00:01+09:00,23379203a894274ce744,4,NaN,NaN,False,0,False,2,Xcode iOS,NaN,iOS Simulatorでシミュレータのsqlite dbを参照する方法,2018-01-01 00:00:01+09:00,https://qiita.com/marlowe/items/23379203a89427...,marlowe
1,False,1,2018-01-01 00:00:06+09:00,020e18563c8616d4bb9a,73,NaN,NaN,False,0,False,73,数学 最適化 シンギュラリティ,NaN,シンギュラリティは近い ～全人類がいますぐ注目すべき特異点論と多目的最適化の知られざる関係～,2023-07-06 22:45:58+09:00,https://qiita.com/hmkz/items/020e18563c8616d4bb9a,hmkz
2,False,1,2018-01-01 00:00:12+09:00,41efb0a5ad4b03542247,2,NaN,NaN,False,0,False,5,Node.js RaspberryPi npm IoT,NaN,IoT開発の利器、pi-sync,2023-11-14 11:20:16+09:00,https://qiita.com/jerrywdlee/items/41efb0a5ad4...,jerrywdlee
3,False,0,2018-01-01 00:00:32+09:00,52dcdd8255f91c927c1e,10,NaN,NaN,False,0,False,4,C# C++CLI mkl 速度比較 .NETCore,NaN,C#でもBLASとかLAPACKを使えるようにした！一部だけ！,2018-02-03 01:43:57+09:00,https://qiita.com/Rafka86/items/52dcdd8255f91c...,Rafka86
4,False,0,2018-01-01 00:03:40+09:00,c3ccd2765f285764e5a0,8,NaN,NaN,False,0,False,7,PHP C++ C# OpenCV Conoha,NaN,C++でマークシート読み取れるフリーソフトを自作してみた！,2018-03-10 19:34:08+09:00,https://qiita.com/takoyaki3/items/c3ccd2765f28...,takoyaki3
5,False,0,2018-01-01 00:11:00+09:00,a1ff5fbbc3b660e01d96,46,NaN,NaN,False,0,False,67,PowerShell InternetExplorer,NaN,PowershellでInternetExplorerを操作する,2019-04-10 13:52:08+09:00,https://qiita.com/flasksrw/items/a1ff5fbbc3b66...,flasksrw
6,False,0,2018-01-01 00:12:37+09:00,90a9052b84745477c96b,4,NaN,NaN,False,0,False,6,GitHub Angular,NaN,Angular CLIでプロジェクトを作成し、GitHubで管理する,2018-01-01 00:41:11+09:00,https://qiita.com/ray6954/items/90a9052b847454...,ray6954
7,False,0,2018-01-01 00:19:58+09:00,294cc728fe744e50d20e,14,yumemi,NaN,False,0,False,7,プログラミング教育 microbit,NaN,6歳にプログラミングを教えて失敗した話,2018-01-01 00:19:58+09:00,https://qiita.com/ikemura23/items/294cc728fe74...,ikemura23
8,False,0,2018-01-01 00:27:21+09:00,539d429729d5819de0aa,394,future,NaN,False,0,False,461,Linux CentOS UNIX ファイルシステム,NaN,知っておきたいLinuxファイルシステムの概念,2018-01-01 00:27:21+09:00,https://qiita.com/tutuz/items/539d429729d5819d...,tutuz
9,False,0,2018-01-01 00:32:18+09:00,77f2072d434f211b26b9,1,NaN,NaN,False,0,False,0,Mac Preview.app Kensington,NaN,[kensington]プレビュー.appのページめくりを楽にする,2018-01-01 00:35:08+09:00,https://qiita.com/ngsw/items/77f2072d434f211b26b9,ngsw


## 分かち書き

In [16]:
# set up
# https://zenn.dev/robes/articles/0b9034cfa3b66d
!apt-get update -y && apt-get -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git gcc g++ curl > /dev/null

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Fetched 119 kB in 1s (156 kB/s)
Reading package lists... Done


In [17]:
!pip install mecab-python3 unidic > /dev/null

# シンボリックリンクによるエラー回避
!ln -s /etc/mecabrc /usr/local/etc/mecabrc
# 辞書のディレクトリを確認
!echo `mecab-config --dicdir`"/unidic"

ln: failed to create symbolic link '/usr/local/etc/mecabrc': File exists
/usr/lib/x86_64-linux-gnu/mecab/dic/unidic


In [18]:
!python -m unidic download

download url: https://cotonoha-dic.s3-ap-northeast-1.amazonaws.com/unidic-3.1.0.zip
Dictionary version: 3.1.0+2021-08-31
unidic-3.1.0.zip: 100%|██████████████████████| 526M/526M [00:27<00:00, 19.0MB/s]
Finished download.
Downloaded UniDic v3.1.0+2021-08-31 to /opt/conda/lib/python3.10/site-packages/unidic/dicdir


In [9]:
df['body'] = df.body.str[:5000]

In [10]:
df

,title,body,tags
0,ios simulatorでシミュレータのsqlite dbを参照する方法,タイトル通り、iosアプリの開発をしていて、ローカルdbの中身を確認したくなった時に使ってま...,Xcode iOS
1,シンギュラリティは近い全人類がいますぐ注目すべき特異点論と多目的最適化の知られざる関係,さて これから解説するシンギュラリティは 0年に人工知能がどうこうという話ではなく 数学にお...,数学 最適化 シンギュラリティ
2,iot開発の利器、pi sync,前書き あけましておめでとうございます 。0年の初qiitaです。よろしくお願いいた...,Node.js RaspberryPi npm IoT
3,c でもblasとかlapackを使えるようにした 一部だけ,新年、あけましておめでとうございます 初投稿になります 皆様はじめまして、rafka...,C# C++CLI mkl 速度比較 .NETCore
4,c でマークシート読み取れるフリーソフトを自作してみた,マークシート等を読み取れるcell sheetというソフトを作成してみました。こんな感じのシ...,PHP C++ C# OpenCV Conoha
...,...,...,...
90308,ipv0マルチキャストによるフェイルオーバー,0 はじめにフェイルオーバー、マスター スレーブ、死活監視構成は、どうしても設計が複雑...,Go Linux Network multicast
90309,充電用のmicrousbケーブルを見つけるデバイス 大晦日ハッカソン,充電用microusbの罠いろいろ基板などにmicrousbがついてきて、気づくと山のよう...,USB 大晦日ハッカソン
90310,luaスクリプトからnetbsd luaカーネルモジュールの値を参照できるようにする,netbsdの intro 0lua には、luaカーネルモジュールが提供する機能が記載...,Lua AdventCalendar NetBSD
90311,python 近くの喫茶店・カフェを教えてくれるline bot作成の記録・解説,そして0年0月末に、動くところまではひっそり完成していました。 blockquote cla...,Python Heroku Python3 linebot LINEmessagingAPI


In [11]:
# 長文をそのままmecab.parseに入力するとエラーになるのである程度の文字数で分割します
# 単語の切れ目などにより単語分割されてしまうこともあるが，許容します
def split_text_into_chunks(text, n):
    chunks = [text[i:i+n] for i in range(0, len(text), n)]
    return chunks

def split_body_into_chunks(df, n):
    df['body_chunks'] = df['body'].apply(lambda x: split_text_into_chunks(x, n))
    return df

# 1000文字ごとにリストに分割
n = 1000
df = split_body_into_chunks(df, n)
df

# もしくは何文字かで区切ります
df['body'] = df.body.str[:5000]

,title,body,tags,body_chunks
0,"[ios, simulator, シミュレータ, sqlite, db, 参照, 方法]",タイトル通り、iosアプリの開発をしていて、ローカルdbの中身を確認したくなった時に使ってま...,Xcode iOS,[タイトル通り、iosアプリの開発をしていて、ローカルdbの中身を確認したくなった時に使って...
1,"[シンギュラリティ, 人類, いま, 注目, 特異, 点, 論, 目的, 関係]",さて これから解説するシンギュラリティは 0年に人工知能がどうこうという話ではなく 数学にお...,数学 最適化 シンギュラリティ,[さて これから解説するシンギュラリティは 0年に人工知能がどうこうという話ではなく 数学に...
2,"[iot, 開発, 利器, pi, sync]",前書き あけましておめでとうございます 。0年の初qiitaです。よろしくお願いいた...,Node.js RaspberryPi npm IoT,[ 前書き あけましておめでとうございます 。0年の初qiitaです。よろしくお願いい...
3,"[blas, lapack, 一部]",新年、あけましておめでとうございます 初投稿になります 皆様はじめまして、rafka...,C# C++CLI mkl 速度比較 .NETCore,[ 新年、あけましておめでとうございます 初投稿になります 皆様はじめまして、rafk...
4,"[マーク, シート, フリー, ソフト, 自作]",マークシート等を読み取れるcell sheetというソフトを作成してみました。こんな感じのシ...,PHP C++ C# OpenCV Conoha,[マークシート等を読み取れるcell sheetというソフトを作成してみました。こんな感じの...
...,...,...,...,...
90308,"[ipv, マルチ, キャスト, フェイルオーバー]",0 はじめにフェイルオーバー、マスター スレーブ、死活監視構成は、どうしても設計が複雑...,Go Linux Network multicast,[ 0 はじめにフェイルオーバー、マスター スレーブ、死活監視構成は、どうしても設計が複...
90309,"[充電, microusb, ケーブル, デバイス, 大晦日, ハッカ, ソン]",充電用microusbの罠いろいろ基板などにmicrousbがついてきて、気づくと山のよう...,USB 大晦日ハッカソン,[ 充電用microusbの罠いろいろ基板などにmicrousbがついてきて、気づくと山のよ...
90310,"[lua, スクリプト, netbsd, lua, カーネル, モジュール, 値, 参照]",netbsdの intro 0lua には、luaカーネルモジュールが提供する機能が記載...,Lua AdventCalendar NetBSD,[netbsdの intro 0lua には、luaカーネルモジュールが提供する機能が記...
90311,"[python, 近く, 喫茶, カフェ, line, bot, 作成, 記録, 解説]",そして0年0月末に、動くところまではひっそり完成していました。 blockquote cla...,Python Heroku Python3 linebot LINEmessagingAPI,[そして0年0月末に、動くところまではひっそり完成していました。 blockquote cl...


In [11]:
df.body[1]

'さて これから解説するシンギュラリティは 0年に人工知能がどうこうという話ではなく 数学における 可微分写像の特異点論 singularity theory of differentiable mappings  です 写像の微分がフルランクでない点 写像が特別な振る舞いをする点を調べる理論です ……待って ブラウザをそっ閉じないで この可微分写像の特異点論こそが「技術的特異点」に出てくる特異点の語源にもなった理論なんです  0  しかも最近になって 多目的最適化と深い関係がある ということが分かってきたため 一部で大変な盛り上がりを見せています  0  そんなわけで この記事では多目的最適化と特異点論の関係について解説します  0    0  あくまでもアナロジーであって 両者の間に科学的な関係はありません   0  一部 僕の周辺ですが   0  この記事を書くために調べて初めて気付いたのですが webで読める多目的最適化の入門記事ってほとんどないんですね 日本オペレーションズ・リサーチ学会の or wiki  に 多目的最適化のページ  くらいでしょうか とはいえこれは「辞典」なので すでに分かっている人向けの読み物な気がします 初学者向けに内容をかみ砕いたストーリー性のある入門記事が増えれば 多目的最適化ももっと流行るはずなのになぁ そんな思いもあってこの記事を書いています   この記事はまだ書きかけです 定理と書いてある内容が平気で間違ってたりしますのでご注意ください 正確なところが知りたい人は文末に挙げた参考文献をあたってください 来年のadvent calendarまでには書き上げたい…    declaremathoperator  rank  rank  declaremathoperator  intr  int  declaremathoperator  cl  cl  declaremathoperator  im  im  declaremathoperator  comp   circ  declaremathoperator  minimize  minimize  declaremathoperator  subjto  subject to  declaremathoperator  st  sush that  newcomma

In [12]:
import re, regex
import MeCab
#Neologdによるトークナイザー（文書ベクトル作成用・わかちだけ）
path = "/usr/lib/x86_64-linux-gnu/mecab/dic/unidic"
mecab = MeCab.Tagger(path)

#正規表現objectの宣言
re_kana = regex.compile(r'[\p{Script=Hiragana}\p{Script=Katakana}ーA-Za-z]+')
re_num = re.compile('[0-9]+')
                     
def mecab_tokenizer(text : str):
    words = []
    tokens = mecab.parse(text).split("\n")[:-1]

    # 自立語 and 名詞のみ表層形を取得
    # https://qiita.com/shimajiroxyz/items/3922d6f7dc8e4b156692
    for t in tokens:
        if '\t' not in t:
            continue

        surface, pos = tuple(t.split('\t'))
        pos = pos.split(',')

        # 数字一文字の除外
        if re_num.fullmatch(surface):
            continue
        #ひらがなまたはカタカナ一文字の除外
        if re_kana.fullmatch(surface) and len(surface) == 1: 
            continue

        if pos[0] == '名詞':
            words.append(surface)

    return words

In [13]:
df['title'] = df['title'].apply(mecab_tokenizer)

In [15]:
df['body'] = df['body'].apply(mecab_tokenizer)

In [16]:
df

,title,body,tags
0,"[ios, simulator, シミュレータ, sqlite, db, 参照, 方法]","[タイトル, 通り, ios, アプリ, 開発, ローカル, db, 中身, 確認, 時, ...",Xcode iOS
1,"[シンギュラリティ, 人類, いま, 注目, 特異, 点, 論, 目的, 関係]","[解説, シンギュラリティ, 年, 人工, 知能, 話, 数学, 可, 微分, 写像, 特異...",数学 最適化 シンギュラリティ
2,"[iot, 開発, 利器, pi, sync]","[前書き, 年, 初, qiita, 今回, テーマ, 最近, npm, アップ, ツール,...",Node.js RaspberryPi npm IoT
3,"[blas, lapack, 一部]","[新年, 初, 投稿, 皆, rafka, 自己, 紹介, 現在, 大学, 数値, シミュレ...",C# C++CLI mkl 速度比較 .NETCore
4,"[マーク, シート, フリー, ソフト, 自作]","[マーク, シート, cell, sheet, ソフト, 作成, 感じ, シート, 端, 全...",PHP C++ C# OpenCV Conoha
...,...,...,...
90308,"[ipv, マルチ, キャスト, フェイルオーバー]","[はじめ, フェイルオーバー, マスター, スレーブ, 死活, 監視, 構成, 設計, 複雑...",Go Linux Network multicast
90309,"[充電, microusb, ケーブル, デバイス, 大晦日, ハッカ, ソン]","[充電, microusb, 罠, 基板, microusb, 山, microusb, 山...",USB 大晦日ハッカソン
90310,"[lua, スクリプト, netbsd, lua, カーネル, モジュール, 値, 参照]","[netbsd, intro, lua, lua, カーネル, モジュール, 提供, 機能,...",Lua AdventCalendar NetBSD
90311,"[python, 近く, 喫茶, カフェ, line, bot, 作成, 記録, 解説]","[年, 月, ところ, 完成, blockquote, class, twitter, tw...",Python Heroku Python3 linebot LINEmessagingAPI


In [17]:
# text情報を連結
df['text'] = df.apply(lambda x : ' '.join(x['title']) + ' ' + ' '.join(x['body']), axis=1)
df = df.drop(['title', 'body'], axis=1)

In [24]:
df = df.drop(['body_chunks'], axis=1)

In [18]:
df[:10]

,tags,text
0,Xcode iOS,ios simulator シミュレータ sqlite db 参照 方法 タイトル 通り i...
1,数学 最適化 シンギュラリティ,シンギュラリティ 人類 いま 注目 特異 点 論 目的 関係 解説 シンギュラリティ 年 人...
2,Node.js RaspberryPi npm IoT,iot 開発 利器 pi sync 前書き 年 初 qiita 今回 テーマ 最近 npm ...
3,C# C++CLI mkl 速度比較 .NETCore,blas lapack 一部 新年 初 投稿 皆 rafka 自己 紹介 現在 大学 数値 ...
4,PHP C++ C# OpenCV Conoha,マーク シート フリー ソフト 自作 マーク シート cell sheet ソフト 作成 感...
5,PowerShell InternetExplorer,powershell internetexplorer 操作 url シェル 取得 shel...
6,GitHub Angular,angular cli プロジェクト 作成 github 管理 os windows bit...
7,プログラミング教育 microbit,プログラミング 失敗 話 もの mac book air usb マウス micro bit...
8,Linux CentOS UNIX ファイルシステム,linux ファイル システム 概念 ファイル システム データ メタ データ 構造 ストレ...
9,Mac Preview.app Kensington,kensington プレビュー app ページ めくり 楽 mac プレビュー app 嫌...


In [63]:
# tsvとして保存
df.to_csv(f'{home}/data/qiita_2018_tokenized.tsv', index=False, sep='\t')

## doc2vec

In [60]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 39.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 951.8 kB/s eta 0:00:00a 0:00:01


In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

l = df['text'].values.tolist()

documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(l)]
model = Doc2Vec(documents, vector_size=100,  window=7, min_count=1, workers=4, epochs=50)

model.save('text-embedding')

In [ ]:
# save embedding
np.save('text-embeddimg-dv-100', model.dv.vectors)

In [35]:
dv_npy = model.dv.vectors
print(dv_npy)

(90313, 100)

In [26]:
df = df.drop(['text'], axis=1)

In [27]:
# tag情報をtsvとして保存
df.to_csv(f'{home}/data/qiita_2018_tag.tsv', index=False, sep='\t')